In [1]:
# first let's load in all our packages
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
import re
import pandas as pd


from mne_bids import (read_raw_bids, BIDSPath, 
                      get_entity_vals, get_datatypes, 
                      make_report)
from mne_bids.stats import count_events

import mne
from mne import make_ad_hoc_cov

basepath = os.path.join(os.getcwd(), "../..")
sys.path.append(basepath)
from mne_hfo import HilbertDetector

In [2]:
%%capture
# this may change depending on where you store the data
root = "C:/Users/patri/Dropbox/fedele_hfo_data"
subjects = get_entity_vals(root, 'subject')
sessions = get_entity_vals(root, 'session')
subjectID = subjects[0]
sessionID = sessions[0]
bids_path = BIDSPath(subject=subjectID, session=sessionID,
                     datatype='ieeg', 
                     suffix='ieeg',
                     extension='.vhdr', root=root)

# get first matching dataset
fpath = bids_path.match()[0]
# load dataset into mne Raw object
extra_params = dict(preload=True)
raw = read_raw_bids(fpath, extra_params)

In [3]:
def convert_to_bipolar(raw, drop_originals=True):
    original_ch_names = raw.ch_names
    ch_names_sorted = sorted(original_ch_names)
    ch_pairs = [] 
    for first, second in zip(ch_names_sorted, ch_names_sorted[1:]):
        firstName = re.sub(r'[0-9]+', '', first)
        secondName = re.sub(r'[0-9]+', '', second)
        if firstName == secondName:
            ch_pairs.append((first,second))
    for ch_pair in ch_pairs:
        raw = mne.set_bipolar_reference(raw, ch_pair[0], ch_pair[1], drop_refs=False)
    if drop_originals:
        raw = raw.drop_channels(original_ch_names)
    return raw

In [4]:
%%capture
raw = convert_to_bipolar(raw)

In [5]:
kwargs = {
    'filter_band': (80, 250), # (l_freq, h_freq)
    'threshold': 3, # Number of st. deviations
    'hfo_name': "ripple",
    'n_jobs': 1,
}
hil_detector = HilbertDetector(**kwargs)

In [8]:
raw.n_times

600000

In [6]:
%%capture
# perform the fits
hil_detector = hil_detector.fit(raw)

In [7]:
hil_detector.chs_hfos_

{'AHR1-AHR2': [[20432, 21978],
  [27998, 29732],
  [47477, 48564],
  [104274, 105898],
  [107579, 109202],
  [147445, 150761],
  [254427, 254843],
  [286952, 288613],
  [324667, 327192],
  [368927, 369451],
  [387318, 389505],
  [391336, 391817],
  [394172, 396368],
  [439560, 441191]],
 'AHR2-AHR3': [[20384, 22113],
  [23288, 24709],
  [34692, 35786],
  [174795, 176584],
  [208312, 209323],
  [219583, 220988],
  [332032, 333525],
  [354569, 355826],
  [367709, 369654],
  [375426, 376355],
  [388154, 391775],
  [498537, 499537],
  [521092, 522148],
  [526450, 526638],
  [533874, 536270],
  [590059, 591227],
  [596841, 598415]],
 'AHR3-AHR4': [[20339, 21918],
  [34326, 35892],
  [61984, 62247],
  [107874, 108709],
  [120051, 120496],
  [122768, 124103],
  [175061, 177481],
  [212071, 213197],
  [237936, 239581],
  [394557, 396020],
  [500594, 501992],
  [508573, 510517],
  [532507, 533738],
  [572129, 572865],
  [596568, 597931]],
 'AHR4-AHR5': [[32104, 33152],
  [175017, 175984],
  [21